<a href="https://colab.research.google.com/github/bygencer3/ErdincGencer_Odev_254329028-/blob/main/YapayZekaPrensipleriFinal_254329028_Erdinc_Gencer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd

# Eğer dosyan doğrudan Excel (.xlsx) formatındaysa:
# df = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/v1_f_IT Department_Sorunlar_.xlsx')

# İlk 5 satırı görerek kontrol edelim
print(df.head())

          Talep Başlığı               Ön Sınıflandırma
0       Bağlantı sorunu                           HATA
1        Donanım talebi  BELİRSİZ (LLM'e Gönderilecek)
2         Erişim hatası                           HATA
3         Erişim talebi  BELİRSİZ (LLM'e Gönderilecek)
4  Hata mesajı alıyorum                           HATA


In [11]:
import re
import pandas as pd
from google.colab import drive

# 1. Google Drive'ı Bağla
drive.mount('/content/drive')

# 2. Dosyayı Oku (Verdiğin yola göre güncellendi)
dosya_yolu = '/content/drive/MyDrive/Colab Notebooks/v1_f_IT Department_Sorunlar_.xlsx'
# Not: Excel dosyasında birden fazla sayfa varsa sheet_name='Mesajlar' gibi ekleme yapabilirsin.
df = pd.read_excel(dosya_yolu)

# 3. Gelişmiş Regex Yapılandırması (re.compile ile yüksek performans)
hata_keywords = ["sorun", "hata", "çalışmıyor", "ulaşılamıyor", "problem", "kesinti", "yanıt vermiyor", "donuyor", "yavaş"]
istek_keywords = ["talep", "istek", "ekleme", "kurulum", "tanımlanması", "yeni kullanıcı", "yetki"]

hata_regex = re.compile(r"|".join(hata_keywords), re.IGNORECASE)
istek_regex = re.compile(r"|".join(istek_keywords), re.IGNORECASE)

def gelismis_siniflandir(metin):
    if pd.isna(metin):
        return "VERI YOK"

    metin = str(metin).lower()

    hata_var = bool(hata_regex.search(metin))
    istek_var = bool(istek_regex.search(metin))

    # Karar Mantığı
    if hata_var and not istek_var:
        return "HATA"
    elif istek_var and not hata_var:
        return "İSTEK"
    else:
        # Hem hata hem istek geçiyorsa veya ikisi de yoksa LLM'e devredilir
        return "BELİRSİZ (LLM ANALİZİ GEREKLİ)"

# 4. Uygulama
# Not: Sütun adın 'body' değilse, aşağıyı dosyadaki gerçek sütun adıyla değiştir.
df['On_Siniflandirma'] = df['body'].apply(gelismis_siniflandir)

# 5. Sonuçları Görselleştir
print("\n--- Sınıflandırma Özeti ---")
print(df['On_Siniflandirma'].value_counts())
print("\n--- İlk 10 Satır Sonucu ---")
print(df[['body', 'On_Siniflandirma']].head(10))



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

--- Sınıflandırma Özeti ---
On_Siniflandirma
HATA                              1486
BELİRSİZ (LLM ANALİZİ GEREKLİ)     865
İSTEK                              649
Name: count, dtype: int64

--- İlk 10 Satır Sonucu ---
                                                body  \
0  Merhaba,\n\nİK portalına ve ERP sistemine girm...   
1  Merhaba,\n\nİK portalına ve ERP sistemine girm...   
2  Selamlar,\n\nVPN'e bağlanmaya çalışırken sürek...   
3  Merhaba,\n\nSAP'a girmeye çalıştığımda sunucu ...   
4  Selamlar,\n\nEkibe yeni bir arkadaşımız katıld...   
5  Selam,\n\nBeni ilgili mail grubuna ekleyebilir...   
6  Selamlar,\n\nVPN'e bağlanmaya çalışırken sürek...   
7  Selam,\n\nUygulamayı açtığımda direkt kapanıyo...   
8  Merhaba,\n\nYeni başlayan personel için CRM si...   
9  Merhaba,\n\nİK portalına ve ERP sistemine girm...   

                 On_Siniflandirma  


In [12]:
df.to_excel('/content/drive/MyDrive/Colab Notebooks/regex_sonuclar.xlsx', index=False)